# SQL Challenge

I decided to tackle this challenge using the Jupyter magics with SQL

### Importing the necessary python packages

In [1]:
import csv
import sqlite3
import sqlalchemy
import pandas as pd

#### Let's first load the SQL extension and establish a connection with the database

In [2]:
%load_ext sql

In [3]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

#### Next we can load the data and insert into the database using pandas

Also let's convert the date times to just dates using pandas already

In [4]:
df = pd.read_csv("Challenge_Files/EKKO.csv")
df['AEDAT'] = pd.to_datetime(df['AEDAT']).dt.date
df.to_sql("EKKO", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("Challenge_Files/EKPO.csv")
df.to_sql("EKPO", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("Challenge_Files/CDHDR.csv")
df['UDATE'] = pd.to_datetime(df['UDATE']).dt.date
df.to_sql("CDHDR", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("Challenge_Files/CDPOS.csv")
df['UDATE'] = pd.to_datetime(df['UDATE']).dt.date
df.to_sql("CDPOS", con, if_exists='replace', index=False, method="multi")


Now with Jupyter SQL magics we can query the databases with SQL commands

In [5]:
%sql SELECT * from EKKO limit 5

 * sqlite:///my_data1.db
Done.


MANDT,EBELN,LIFNR,BUKRS,BSTYP,BSART,LOEKZ,STATU,AEDAT
10,71160672,65661,15,F,NB,None,9.0,2020-01-29
10,71161696,66597,10,F,NB,None,9.0,2020-05-11
10,71160694,5122,10,F,NB,None,9.0,2020-01-31
10,71160997,62299,15,F,NB,None,9.0,2020-03-16
10,71161765,56690,10,F,NB,None,9.0,2020-07-13


In [6]:
%sql SELECT * from CDPOS limit 5

 * sqlite:///my_data1.db
Done.


MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,TABNAME,TABKEY,FNAME,CHNGIND,VALUE_NEW,VALUE_OLD,UDATE,UTIME
10,EINKBELEG,4700001106,51867463,EKPO,10470000110600030,NETPR,U,25000.00,15000.00,2018-01-08,1970-01-01T15:54:10
10,EINKBELEG,4700001106,51867463,EKPO,10470000110600030,NETWR,U,25000.00,15000.00,2018-01-08,1970-01-01T15:54:10
10,EINKBELEG,4700001106,51873447,EKPO,10470000110600030,NETPR,U,50000.00,25000.00,2018-01-09,1970-01-01T08:22:23
10,EINKBELEG,4700001106,51873447,EKPO,10470000110600030,NETWR,U,50000.00,25000.00,2018-01-09,1970-01-01T08:22:23
10,EINKBELEG,4700001106,51874715,EKPO,10470000110600030,NETPR,U,25000.00,50000.00,2018-01-09,1970-01-01T09:24:08


### Input parameters

User provided input parameters: MANDT, EBELN and D_DATE. We can either prompt for these parameters, or for the purposes of this task just set them manually

In [7]:
#MANDT = input("Enter MANDT: ")
#EBELN = input("Enter EBELN: ")
#D_DATE = input("Enter D_DATE (YYYY-MM-DD): ")

MANDT = "010"
EBELN = "0076052494"
D_DATE = "2018-02-25"

## Query to get the total net worth with the input parameters

The two tabels EKKO and EKPO are joined together based on the MANDT and EBELN values. Then we calculate the current total net order value by taking the sum of all net oder values with the user defined MANDT and EBELN values up to the date provided also by the user.

In [8]:
%sql select SUM(NETWR) as TOTAL_NETWR \
from EKPO join EKKO on EKPO.MANDT = EKKO.MANDT and EKPO.EBELN = EKKO.EBELN \
where EKKO.MANDT = :MANDT and EKKO.EBELN = :EBELN and EKKO.AEDAT <= :D_DATE

 * sqlite:///my_data1.db
Done.


TOTAL_NETWR
950.0


The price of the item ordered might have changed and we want the latest value up to the date provided by the user. So we need to check the database with the price changes recorded and take the old networth with the correct MANDT and EBELN values, and the date after the order date and up to the user provided date. 

In [9]:
%sql select VALUE_OLD from CDPOS \
    where MANDANT = :MANDT and OBJECTID = :EBELN and \
    UDATE > (select AEDAT from EKKO where EKKO.MANDT = :MANDT and EKKO.EBELN = :EBELN) and UDATE <= :D_DATE \
    and FNAME = "NETWR" \
    ORDER BY UDATE, UTIME asc limit 1

 * sqlite:///my_data1.db
Done.


VALUE_OLD
850.00
